---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File ~\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:3805, in Index.get_loc(
    self=Index(['stage_detail', 'stage_name', 'stage_numb...escription', 'id', 'type'],
      dtype='object'),
    key='total'
)
   3804 try:
-> 3805     return self._engine.get_loc(casted_key)
        casted_key = 'total'
        self = Index(['stage_detail', 'stage_name', 'stage_number', 'stage_status', 'title',
       'date_created', 'date_lost', 'date_won', 'description', 'id', 'type'],
      dtype='object')
   3806 except KeyError as err:

File index.pyx:167, in pandas._libs.index.IndexEngine.get_loc()

File index.pyx:196, in pandas._libs.index.IndexEngine.get_loc()

File pandas\\_libs\\hashtable_class_helper.pxi:7081, in pandas._libs.hashtable.PyObjectHashTable.get_item()

File pandas\\_libs\\hashtable_class_helper.pxi:7089, in pandas._libs.hash

In [291]:
import pandas as pd
import requests
import json

API_KEY = '881078-ec5446d8-7fbd-4fac-806d-8a4d81eece36'

next_url = "https://api.agendor.com.br/v3/deals"

# Cabeçalhos da requisição
headers = {
    "Authorization": f"Token {API_KEY}",
    "Content-Type": "application/json"
}

params={"per_page": 100}

all_json_deals_data = []

while next_url:
    response = requests.get(next_url, headers=headers, params=params)
    json_deal_data = response.json()['data']
    all_json_deals_data.extend(json_deal_data)
    links = response.json()['links']
    if 'next' in links.keys():
        next_url = links['next']
    else:
        next_url = False


# Salvando como JSON
with open("deals.json", "w", encoding="utf-8") as arquivo:
    json.dump(all_json_deals_data, arquivo, indent=4, ensure_ascii=False)

# Ler arquivo JSON com vários objetos
with open("deals.json", "r", encoding="utf-8") as arquivo:
    deals = json.load(arquivo)

deal_by_stage = {
    'stage_detail': [], 'stage_name': [], 'stage_number': [], 'stage_status': [],
    'person': [], 'title': [], 'date_created': [], 'date_lost': [], 'date_won': [],
    'organization': [], 'description': []
}
for deal in deals:
    stage_detail = deal['dealStage']['name']
    stage_number = int(deal['dealStage']['sequence'])
    stage_name = deal['dealStage']['funnel']['name']
    stage_status = deal['dealStatus']['name']
    if deal['person']:
        person = deal['person']['id']
    else:
        person = None
    if deal['organization']:
        organization = deal['organization']['id']
    else:
        organization = None
    date_created = deal['createdAt']
    date_won = deal['wonAt']
    date_lost = deal['lostAt']
    description = deal['description']
    title = deal['title']
    deal_by_stage['stage_detail'].append(stage_detail)
    deal_by_stage['stage_number'].append(stage_number)
    deal_by_stage['stage_name'].append(stage_name)
    deal_by_stage['stage_status'].append(stage_status)
    deal_by_stage['person'].append(person)
    deal_by_stage['title'].append(title)
    deal_by_stage['date_created'].append(date_created)
    deal_by_stage['date_lost'].append(date_lost)
    deal_by_stage['date_won'].append(date_won)
    deal_by_stage['organization'].append(organization)
    deal_by_stage['description'].append(description)

pd.DataFrame(deal_by_stage).to_csv('deal_by_stage.csv', index=False, encoding='utf-8')

In [293]:
import json

# Salvando como JSON
with open("deals.json", "w", encoding="utf-8") as arquivo:
    json.dump(all_json_deals_data, arquivo, indent=4, ensure_ascii=False)

In [2]:
import json

# Ler arquivo JSON com vários objetos
with open("deals.json", "r", encoding="utf-8") as arquivo:
    deals = json.load(arquivo)

In [9]:
titles = {'title': []}
for deal in deals:
    titles['title'].append(deal['title'])

In [25]:
df = pd.DataFrame(titles)

In [231]:
pd.DataFrame(deal_by_stage).to_csv('deal_by_stage.csv', index=False, encoding='utf-8')

In [229]:
deal_by_stage = {
    'stage_detail': [], 'stage_name': [], 'stage_number': [], 'stage_status': [],
    'person': [], 'title': [], 'date_created': [], 'date_lost': [], 'date_won': [],
    'organization': [], 'description': []
}
for deal in deals:
    stage_detail = deal['dealStage']['name']
    stage_number = int(deal['dealStage']['sequence'])
    stage_name = deal['dealStage']['funnel']['name']
    stage_status = deal['dealStatus']['name']
    if deal['person']:
        person = deal['person']['id']
    else:
        person = None
    if deal['organization']:
        organization = deal['organization']['id']
    else:
        organization = None
    date_created = deal['createdAt']
    date_won = deal['wonAt']
    date_lost = deal['lostAt']
    description = deal['description']
    title = deal['title']
    deal_by_stage['stage_detail'].append(stage_detail)
    deal_by_stage['stage_number'].append(stage_number)
    deal_by_stage['stage_name'].append(stage_name)
    deal_by_stage['stage_status'].append(stage_status)
    deal_by_stage['person'].append(person)
    deal_by_stage['title'].append(title)
    deal_by_stage['date_created'].append(date_created)
    deal_by_stage['date_lost'].append(date_lost)
    deal_by_stage['date_won'].append(date_won)
    deal_by_stage['organization'].append(organization)
    deal_by_stage['description'].append(description)


    

In [137]:
from datetime import datetime, timedelta

In [139]:
def process_data_for_line_plot(df):
    df['id'] = df[



In [309]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
from datetime import datetime, timedelta

In [387]:
df = pd.DataFrame(deal_by_stage)


# Supondo que df já esteja carregado
# Criar as colunas 'id' e 'type'
df['id'] = df['person'].fillna(df['organization'])
df['type'] = df['person'].apply(lambda x: 'person' if pd.notna(x) else 'organization')

# Remover as colunas 'person' e 'organization'
df = df.drop(columns=['person', 'organization'])

# Remover números no início da coluna 'stage_name'
df['stage_name'] = df['stage_name'].str.replace(r'^\d+\s*', '', regex=True)

# Filtragem por data
df['date_created'] = pd.to_datetime(df['date_created']).dt.date

# Transformação da coluna 'stage_detail'
df['stage_detail'] = df['stage_detail'].replace({
    'CONTATO': '1.1 LEADS',
    'TYPEFORM': '2.1 VALIDAÇÃO',
    'CONTRATO': '3.1 ATIVOS'
})

final_df = None

# Link externo para CSS
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Criar o aplicativo Dash
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Dashboard de Stage Detail"

# Valor inicial do filtro de data
default_start_date = datetime.today() - timedelta(days=60)
default_end_date = datetime.today()

# Layout do dashboard
app.layout = html.Div(children=[
    html.H1(children="Dashboard de Stage Detail", style={'text-align': 'center', 'color': '#003366'}),

    # Container para os filtros dinâmicos
    html.Div(id='date-filters-container', children=[
        html.Div([
            html.Label(f'Filtro 1', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': 0},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
    ]),

    # Botão para adicionar filtros
    html.Button("Adicionar Filtro", id="add-filter-btn", n_clicks=0, style={'background-color': '#003366', 'color': 'white'}),

    # Gráfico
    dcc.Graph(id='line-chart'),

    # Gráfico de barras
    dcc.Graph(
        id='bar-chart',
        figure=px.bar(
            df_bar, x='stage_detail', y='count', text='count',
            title="Visão Geral por Stage Detail"
        ).update_traces(
            texttemplate='%{text}', textposition='outside'
        ).update_layout(
            uniformtext_minsize=8, uniformtext_mode='hide',
            plot_bgcolor='white', paper_bgcolor='rgba(0,0,0,0)',
            font={'color': '#003366'}
        )
    )
    
], style={'font-family': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#FFFFFF'})

# Callback para adicionar novos filtros dinamicamente
@app.callback(
    Output('date-filters-container', 'children'),
    Input('add-filter-btn', 'n_clicks'),
    State('date-filters-container', 'children')
)
def add_date_filter(n_clicks, children):
    if n_clicks > 0:
        new_filter_index = n_clicks + 1
        new_filter = html.Div([
            html.Label(f'Filtro {new_filter_index}', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': n_clicks},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
        children.append(new_filter)
    return children

# Callback para atualizar o gráfico baseado nos filtros aplicados
@app.callback(
    Output('line-chart', 'figure'),
    [Input({'type': 'date-filter', 'index': dash.ALL}, 'start_date'),
     Input({'type': 'date-filter', 'index': dash.ALL}, 'end_date')]
)
def update_chart(start_dates, end_dates):
    global final_df
    fig = px.line(title="Evolução por Stage Detail")
    if not start_dates or not end_dates:
        return fig

    for i, (start_date, end_date) in enumerate(zip(start_dates, end_dates)):
        if start_date and end_date:
            start_date = datetime.fromisoformat(start_date).date()
            end_date = datetime.fromisoformat(end_date).date()
            filtered_df = df[(df['date_created'] >= start_date) & (df['date_created'] <= end_date)]
            filtered_df = filtered_df.loc[filtered_df.groupby('id')['date_created'].idxmax()]


            filtered_df = filtered_df.groupby('stage_detail').size().reset_index(name='total')
            
            fig.add_scatter(x=filtered_df['stage_detail'], y=filtered_df['total'], mode='lines+markers', name=f'Filtro {i + 1}')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font={'color': '#003366'})
    return fig




# Rodar o app
if __name__ == '__main__':
    app.run_server(debug=True)

In [415]:
df = pd.DataFrame(deal_by_stage)


# Supondo que df já esteja carregado
# Criar as colunas 'id' e 'type'
df['id'] = df['person'].fillna(df['organization'])
df['type'] = df['person'].apply(lambda x: 'person' if pd.notna(x) else 'organization')
df = df[~df['stage_detail'].str.contains("5|6", na=False)]


# Remover as colunas 'person' e 'organization'
df = df.drop(columns=['person', 'organization'])

# Remover números no início da coluna 'stage_name'
df['stage_name'] = df['stage_name'].str.replace(r'^\d+\s*', '', regex=True)

# Filtragem por data
df['date_created'] = pd.to_datetime(df['date_created']).dt.date

# Transformação da coluna 'stage_detail'
df['stage_detail'] = df['stage_detail'].replace({
    'CONTATO': '1.1 LEADS',
    'TYPEFORM': '2.1 VALIDAÇÃO',
    'CONTRATO': '3.1 ATIVOS'
})

df_line = df

df = pd.DataFrame(deal_by_stage)


# Supondo que df já esteja carregado
# Criar as colunas 'id' e 'type'
df['id'] = df['person'].fillna(df['organization'])
df['type'] = df['person'].apply(lambda x: 'person' if pd.notna(x) else 'organization')

# Remover as colunas 'person' e 'organization'
df = df.drop(columns=['person', 'organization'])

# Remover números no início da coluna 'stage_name'
df['stage_name'] = df['stage_name'].str.replace(r'^\d+\s*', '', regex=True)

# Filtragem por data
df['date_created'] = pd.to_datetime(df['date_created']).dt.date


df_filtered = df[(df['stage_name'] == 'AMBULANTE ESSENCIAL') | df['description'].str.contains("CA", na=False)]


dt_filtered = df_filtered.loc[df_filtered.groupby('id')['date_created'].idxmax()]
df_filtered = df.dropna(subset=['id'])

dt_filtered = dt_filtered[~dt_filtered['stage_detail'].str.contains('1', na=False)]
d = dt_filtered['stage_detail'].value_counts()

df_bar = d.reset_index()
df_bar.columns = ['stage_detail', 'count']


# Link externo para CSS
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Criar o aplicativo Dash
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "Dashboard de Stage Detail"

# Valor inicial do filtro de data
default_start_date = datetime.today() - timedelta(days=60)
default_end_date = datetime.today()

# Layout do dashboard
app.layout = html.Div(children=[
    html.H1(children="Dashboard de Stage Detail", style={'text-align': 'center', 'color': '#003366'}),

    # Container para os filtros dinâmicos
    html.Div(id='date-filters-container', children=[
        html.Div([
            html.Label(f'Filtro 1', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': 0},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
    ]),

    # Botão para adicionar filtros
    html.Button("Adicionar Filtro", id="add-filter-btn", n_clicks=0, style={'background-color': '#003366', 'color': 'white'}),

    # Gráfico
    dcc.Graph(id='line-chart'),

    # Gráfico de barras
    dcc.Graph(
        id='bar-chart',
        figure=px.bar(
            df_bar, x='stage_detail', y='count', text='count',
            title="Visão Geral por Stage Detail"
        ).update_traces(
            texttemplate='%{text}', textposition='outside'
        ).update_layout(
            uniformtext_minsize=8, uniformtext_mode='hide',
            plot_bgcolor='white', paper_bgcolor='rgba(0,0,0,0)',
            font={'color': '#003366'}
        )
    )
    
], style={'font-family': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#FFFFFF'})

# Callback para adicionar novos filtros dinamicamente
@app.callback(
    Output('date-filters-container', 'children'),
    Input('add-filter-btn', 'n_clicks'),
    State('date-filters-container', 'children')
)
def add_date_filter(n_clicks, children):
    if n_clicks > 0:
        new_filter_index = n_clicks + 1
        new_filter = html.Div([
            html.Label(f'Filtro {new_filter_index}', style={'color': '#003366'}),
            dcc.DatePickerRange(
                id={'type': 'date-filter', 'index': n_clicks},
                min_date_allowed=df['date_created'].min(),
                max_date_allowed=df['date_created'].max(),
                start_date=default_start_date,
                end_date=default_end_date,
                display_format='DD/MM/YYYY',
                style={'margin-bottom': '10px'}
            )
        ], style={'border': '2px solid #003366', 'padding': '10px', 'border-radius': '5px'})
        children.append(new_filter)
    return children

# Callback para atualizar o gráfico baseado nos filtros aplicados
@app.callback(
    Output('line-chart', 'figure'),
    [Input({'type': 'date-filter', 'index': dash.ALL}, 'start_date'),
     Input({'type': 'date-filter', 'index': dash.ALL}, 'end_date')]
)
def update_chart(start_dates, end_dates):
    global final_df
    fig = px.line(title="Evolução por Stage Detail")
    if not start_dates or not end_dates:
        return fig

    for i, (start_date, end_date) in enumerate(zip(start_dates, end_dates)):
        if start_date and end_date:
            start_date = datetime.fromisoformat(start_date).date()
            end_date = datetime.fromisoformat(end_date).date()
            filtered_df = df_line[(df_line['date_created'] >= start_date) & (df['date_created'] <= end_date)]
            filtered_df = filtered_df.loc[filtered_df.groupby('id')['date_created'].idxmax()]


            filtered_df = filtered_df.groupby('stage_detail').size().reset_index(name='total')
            
            fig.add_scatter(x=filtered_df['stage_detail'], y=filtered_df['total'], mode='lines+markers', name=f'Filtro {i + 1}')
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)', font={'color': '#003366'})
    return fig




# Rodar o app
if __name__ == '__main__':
    app.run_server(debug=True)

In [389]:
df = pd.DataFrame(deal_by_stage)


# Supondo que df já esteja carregado
# Criar as colunas 'id' e 'type'
df['id'] = df['person'].fillna(df['organization'])
df['type'] = df['person'].apply(lambda x: 'person' if pd.notna(x) else 'organization')

# Remover as colunas 'person' e 'organization'
df = df.drop(columns=['person', 'organization'])

# Remover números no início da coluna 'stage_name'
df['stage_name'] = df['stage_name'].str.replace(r'^\d+\s*', '', regex=True)

# Filtragem por data
df['date_created'] = pd.to_datetime(df['date_created']).dt.date


df_filtered = df[(df['stage_name'] == 'AMBULANTE ESSENCIAL') | df['description'].str.contains("CA", na=False)]


dt_filtered = df_filtered.loc[df_filtered.groupby('id')['date_created'].idxmax()]
df_filtered = df.dropna(subset=['id'])

dt_filtered = dt_filtered[~dt_filtered['stage_detail'].str.contains('1', na=False)]
d = dt_filtered['stage_detail'].value_counts()

df_bar = d.reset_index()
df_bar.columns = ['stage_detail', 'count']



In [395]:
df

,stage_detail,stage_name,stage_number,stage_status,title,date_created,date_lost,date_won,description,id,type
0,2.1 VALIDAÇÃO,SERVICE,1,Ganho,Andre Dias de Matos - 02,2025-01-24,None,2025-02-04T20:33:52.000Z,None,55574570.0,person
1,3.1 NOVOS ATIVOS,CUSTOMER SUCCESS,1,Em andamento,9068 - Andre Dias de Matos - 02,2025-02-04,None,None,None,55574570.0,person
2,2.3 VINCULAÇÃO,SERVICE,3,Em andamento,8731 - Amanda da Silva Oliveira,2025-01-08,None,None,None,55575829.0,person
3,2.1 VALIDAÇÃO,SERVICE,1,Perdido,Amanda,2025-01-07,2025-02-04T19:38:43.000Z,None,None,57020920.0,person
4,2.3 VINCULAÇÃO,SERVICE,3,Em andamento,8305 - Mauro Cesar,2024-11-29,None,None,None,55576306.0,person
...,...,...,...,...,...,...,...,...,...,...,...
7221,1.1 LEADS,POS,1,Em andamento,241 - +5541998240043,2024-10-25,None,None,None,NaN,organization
7222,1.1 LEADS,POS,1,Em andamento,239 - +5598981971201,2024-10-25,None,None,None,NaN,organization
7223,1.1 LEADS,POS,1,Em andamento,237 - Abadia Tavares,2024-10-25,None,None,None,NaN,organization
7224,1.1 LEADS,POS,1,Em andamento,234 - +5582996406892,2024-10-25,None,None,None,NaN,organization


In [399]:
df = df[~df['stage_detail'].str.contains("5", na=False)]


In [401]:
df

,stage_detail,stage_name,stage_number,stage_status,title,date_created,date_lost,date_won,description,id,type
0,2.1 VALIDAÇÃO,SERVICE,1,Ganho,Andre Dias de Matos - 02,2025-01-24,None,2025-02-04T20:33:52.000Z,None,55574570.0,person
1,3.1 NOVOS ATIVOS,CUSTOMER SUCCESS,1,Em andamento,9068 - Andre Dias de Matos - 02,2025-02-04,None,None,None,55574570.0,person
2,2.3 VINCULAÇÃO,SERVICE,3,Em andamento,8731 - Amanda da Silva Oliveira,2025-01-08,None,None,None,55575829.0,person
3,2.1 VALIDAÇÃO,SERVICE,1,Perdido,Amanda,2025-01-07,2025-02-04T19:38:43.000Z,None,None,57020920.0,person
4,2.3 VINCULAÇÃO,SERVICE,3,Em andamento,8305 - Mauro Cesar,2024-11-29,None,None,None,55576306.0,person
...,...,...,...,...,...,...,...,...,...,...,...
7221,1.1 LEADS,POS,1,Em andamento,241 - +5541998240043,2024-10-25,None,None,None,NaN,organization
7222,1.1 LEADS,POS,1,Em andamento,239 - +5598981971201,2024-10-25,None,None,None,NaN,organization
7223,1.1 LEADS,POS,1,Em andamento,237 - Abadia Tavares,2024-10-25,None,None,None,NaN,organization
7224,1.1 LEADS,POS,1,Em andamento,234 - +5582996406892,2024-10-25,None,None,None,NaN,organization
